# Install library

In [11]:
%pip install urllib3
%pip install Pillow
%pip install opencv-python
%pip install requests
%pip install selenium
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Import library

In [12]:
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import sys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import csv
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
import os

# Load config

In [13]:
load_dotenv('.env')

URL_BASE = os.getenv('URL_BASE')
URL = os.getenv('URL')
URL_E = os.getenv('URL_E')

# Functions

In [14]:
# get book with image[0] have white border
def filter_book_image(url: str) -> bool:
    np.set_printoptions(threshold=sys.maxsize)

    response = requests.get(url)

    image = Image.open(BytesIO(response.content))
    arr = np.asarray(image)
    t = [[255,255,255]]*50
    
    return np.sum(arr[0][:50]) != np.sum(t)

In [15]:
# save to file csv
def save_file(name_file,header, data):
    with open(name_file+'.csv', 'w') as file:
        # 2. Create a CSV writer
        writer = csv.writer(file)
        # 3. Write data to the file
        writer.writerow(header)
        writer.writerows(data)

In [16]:
def get_reviews(driver):
    # time.sleep(0.5)

    reviews = []

    ratings = driver.find_elements(By.XPATH, '//meta[@itemprop="ratingValue"]')
    
    comments = driver.find_elements(By.XPATH, '//h3[@itemprop="headline"]')

    data_time = driver.find_elements(By.XPATH, '//meta[@itemprop="dateCreated"]')
    
    liked = driver.find_elements(By.XPATH, '//button[@class="bv-content-btn bv-content-btn-feedback-yes bv-focusable"]/span/span[@class="bv-content-btn-count"]')
    
    disliked = driver.find_elements(By.XPATH, '//button[@class="bv-content-btn bv-content-btn-feedback-no bv-focusable"]/span/span[@class="bv-content-btn-count"]')
    
    for i in range(len(ratings)):
        review = [ratings[i].get_attribute("content"), 
                  comments[i].get_attribute("innerHTML"), 
                  liked[i].get_attribute("innerHTML"), 
                  disliked[i].get_attribute("innerHTML"), 
                  data_time[i].get_attribute("content")]
        reviews.append(review)
    
    return reviews

In [17]:
def get_data(driver):
    book_title = driver.find_element(By.XPATH, '//h1[@itemprop="name"]').get_attribute("innerHTML")
    
    pushlisher = driver.find_element(By.XPATH,'//span[@itemprop="publisher"]').get_attribute("innerHTML")
    author = driver.find_element(By.XPATH,'//span[@id="key-contributors"]//a').get_attribute("innerHTML")
    
    price = driver.find_element(By.XPATH,'//span[@id="pdp-cur-price"]').get_attribute("innerHTML")
    # if not price:
    #     price = driver.find_element(By.XPATH,'//div[@id="catalog-product-details-price"]//p[@class="special-price"]/span[@class="price"]').text
    
    category_list = driver.find_elements(By.XPATH,'//div[@class="row related-subject-container"]/div/div/span/a')
    categories = [cat.get_attribute("innerHTML") for cat in category_list]
    # category = [categories[1], categories[2]]
        
    rating = driver.find_element(By.XPATH,'//div[@class="bv_numReviews_text"]').get_attribute("innerHTML")
    if rating != "(0)":
        rating = driver.find_element(By.XPATH,'//div[@class="bv_avgRating_component_container notranslate"]').get_attribute("innerHTML")
    
    main_image = driver.find_element(By.XPATH,'//img[@id="pdpMainImage"]')
    image_list = driver.find_elements(By.XPATH,'//img[@class="ResolveComplete"]')
    images = [main_image.get_attribute("src")]
    images.extend([image.get_attribute("src") for image in image_list])
    
    desc = driver.find_element(By.XPATH,'//div[@class="text--medium overview-content p-lg-4 p-sm-0 bookseller-cont"]').get_attribute("innerHTML")
    description=desc.replace("\n","<br/>")
    
    plain_title = driver.find_elements(By.XPATH,'//table[@class="plain centered"]/tbody/tr/th')
    plain = driver.find_elements(By.XPATH,'//table[@class="plain centered"]/tbody/tr/td')
    
    publication_date = ""
    page = ""
    product_dimensions = ""
    
    for i in range(len(plain_title)):
        if plain_title[i].get_attribute("innerHTML") == 'Publication date:':
            publication_date = plain[i].get_attribute("innerHTML")
        if plain_title[i].get_attribute("innerHTML") == 'Pages:':
            page = plain[i].get_attribute("innerHTML")
        if plain_title[i].get_attribute("innerHTML") == 'Product dimensions:':
            product_dimensions = plain[i].get_attribute("innerHTML")
    
    return [book_title,
            pushlisher,
            author,
            price,
            categories,
            rating, 
            images,
            description,
            publication_date,
            page,
            product_dimensions]

# Get data

## Create driver chrome

In [18]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--start-maximized")

driver=webdriver.Chrome(options=chrome_options)

driver.get(URL)

## Get data raw

In [19]:
fields = ['book_title',
            'pushlisher',
            'author',
            'price',
            'category',
            'rating', 
            'images',
            'desc',
            'publication_date',
            'page',
            'product_dimensions',
            'rating_count',
            'reviews']

pagination = driver.find_elements(By.XPATH, '//ul[@role="navigation"]/li')
max_pag =  int(pagination[len(pagination)-2].text.strip().replace("Page","").replace("\\n",""))
print(max_pag)
# max_pag =  100
# max_pag =  1
data = []

for pag in range(1,max_pag+1):
    url_pag = URL_E + str(pag)
    driver.get(url_pag)
    
    books = driver.find_elements(By.XPATH, '//div[@class="product-image-container plp-quick-add-cont"]/div/a')
    
    book_url = [book_url.get_attribute("data-quickview-url") for book_url in books]
    
    for e in book_url:
        try:
            driver.get(URL_BASE + e)
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            # time.sleep(1)
            
            data_book = get_data(driver)
                
            reviews=[]
            
            if data_book[5] != "(0)":
                review_end = get_reviews(driver)
                if len(review_end) > 8:
                    pag_reviews=driver.find_element(By.XPATH, '//li[@class="bv-content-pagination-buttons-item bv-content-pagination-buttons-item-next"]//a')
                    check_pag_reviews = pag_reviews.get_attribute("href")
                    reviews=[review_end]
                
                    while check_pag_reviews:
                        driver.get(check_pag_reviews)
                        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
                    
                        review_end = get_reviews(driver)
                        
                        reviews.extend(review_end)
                    
                        pag_reviews=driver.find_element(By.XPATH, '//li[@class="bv-content-pagination-buttons-item bv-content-pagination-buttons-item-next"]')
                        check_pag_reviews = pag_reviews.get_attribute("href")
                else:
                    reviews.extend(review_end)

            # else:
            #     review_end = get_reviews(driver)
            #     reviews.extend(review_end)
            
            # if filter_book_image(data_book[6][0]) and data_book[4][0] != 'OTHER LANGUAGES':
            data_book.append(len(reviews))
            data_book.append(reviews)
            data.append(data_book)
            
            save_file('data_raw/books_raw',fields,data)
        except Exception as e:
            print(e)
        
    if len(data) % 2000 == 0:
        print(data)
        
    if len(data) >= 10000:
            break

50
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="text--medium overview-content p-lg-4 p-sm-0 bookseller-cont"]"}
  (Session info: chrome=117.0.5938.88); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x55939efc36c3 <unknown>
#1 0x55939ec991e7 <unknown>
#2 0x55939ece0566 <unknown>
#3 0x55939ece0651 <unknown>
#4 0x55939ed1da74 <unknown>
#5 0x55939ed020ed <unknown>
#6 0x55939ed1b322 <unknown>
#7 0x55939ed01e93 <unknown>
#8 0x55939ecd4934 <unknown>
#9 0x55939ecd571e <unknown>
#10 0x55939ef88cc8 <unknown>
#11 0x55939ef8cc00 <unknown>
#12 0x55939ef971ac <unknown>
#13 0x55939ef8d818 <unknown>
#14 0x55939ef5a28f <unknown>
#15 0x55939efb1e98 <unknown>
#16 0x55939efb2069 <unknown>
#17 0x55939efc2853 <unknown>
#18 0x7f0f91694b43 <unknown>

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="te

## Close driver

In [20]:
driver.close()